# CAMs Generations 

Because our methods are globally interpretable, we generate the CAMS to be stored persistently.

In [ ]:
!pip install wandb --quiet 
!pip install -U git+https://github.com/aguirrejuan/ConvRFF.git@2d8b48f9f3825192585430fa3285b675d263e560 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
import os 
import numpy as np 

import wandb 
from convRFF.models import load_model

from convRFF.data import get_data
from gcpds.image_segmentation.datasets.segmentation import InfraredThermalFeet

from convRFF.class_activation_maps import gen_calculate
from convRFF.class_activation_maps import save_mimic_mmap
from convRFF.class_activation_maps import load_mimic_mmap

from tf_keras_vis.layercam import Layercam
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
from gcpds.image_segmentation.class_activation_maps import SegScore

In [ ]:
def load_model_from_run(run, model_name= 'model-best.h5'):
    path = run.file(model_name).download(replace=True).name
    model = load_model(path)
    return model

def calculate_and_save_cam_data(cam_method, model, data, file_path, batch_size=32, targe_classes=[0,1], model_name=None):
    layers = [layer.name for layer in model.layers if 'Conv' in layer.name or 'Line' in layer.name]
    if model_name.startswith('f'):
        layers.extend(['Pool30','Pool40'])
    elif model_name.startswith('r'):
        layers.extend(['Res01_Act01','Res03_Act01','Res05_Act01','Res07_Act01'])
    print(layers)
    total_rows = len(layers)
    generator = gen_calculate(cam_method, layers, data.batch(batch_size), targe_classes)
    save_mimic_mmap(generator, file_path, dtype=DTYPE)

def get_path_run(run,root_path):
    run_id = '_'.join([*run.path,run.config['model']])
    folder_path = os.path.join(root_path, run_id)
    return folder_path

def cam_data(run, cam_class, data, root_path,targe_classes, batch_size=32, 
                name_file='cam_data.mimic_memmap', rewrite=False):
    model = load_model_from_run(run) 
    cam_method = cam_class(model,
                           model_modifier=ReplaceToLinear(),
                            clone=True)
    folder_path = get_path_run(run,root_path)
    os.makedirs(folder_path, exist_ok=True)
    file_path = os.path.join(folder_path,name_file)

    if rewrite or not os.path.exists(file_path):
        calculate_and_save_cam_data(cam_method, model, data, file_path ,batch_size=batch_size,targe_classes=targe_classes, model_name=run.config['model'])
    

In [ ]:
WANDB_PROJECT = 'thesis_experiments'
ROOT_FOLDER = 'ROOT_FOLDER_TO_SAVE_CAMS'

api = wandb.Api()
runs = api.runs(WANDB_PROJECT)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import tensorflow as tf


cam_class = Layercam
root_path = ROOT_FOLDER

datasets = ['infrared_thermal_feet_nodataaug', 'infrared_thermal_feet']

for dataset in datasets: 
    if dataset == 'infrared_thermal_feet':
        data, *_ = get_data(InfraredThermalFeet, data_augmentation=False, return_label_info=True,shape=224, split=[0.1,0.1])
        data = data.unbatch()
        name_file = 'infrared_thermal_feet_layercam_mimic_memmap'
        targe_classes = [0,1]
        ids = [run.id for run in runs if  run.state=='finished' and run.config['dataset'] == 'infrared_thermal_feet' and run.sweep.id == '6l60ok26' and (run.config['model'] !=  'u_r_b')]
        DTYPE = np.dtype([('info_instance', 'U50', (134,2)),  
                    ('cam', np.float32, (134,224, 224, 2))
                    ])
        
    elif dataset == 'infrared_thermal_feet_nodataaug':
        data, *_ = get_data(InfraredThermalFeet, data_augmentation=False, return_label_info=True,shape=224, split=[0.1,0.1])
        data = data.unbatch()
        name_file = 'infrared_thermal_feet_layercam.mimic_memmap'
        targe_classes = [0,1]
        ids = [run.id for run in runs if  run.state=='finished' and run.config['dataset'] == 'infrared_thermal_feet_nodataaug' and run.sweep.id == 'hg11kida' and (run.config['model'] !=  'u_r_b')]
        DTYPE = np.dtype([('info_instance', 'U50', (134,2)),  
                    ('cam', np.float32, (134,224, 224, 2))
                    ]) 


    for id_ in ids:
        print(id_)
        run = [run for run in runs if run.id ==id_][0]
        cam_data(run, cam_class, data, root_path,targe_classes, batch_size=32, name_file=name_file, rewrite=True)

hz0pen55
['Conv10', 'Conv11', 'Conv20', 'Conv21', 'Conv30', 'Conv31', 'Conv40', 'Conv41', 'Conv50', 'Conv51', 'Conv04', 'Conv60', 'Conv61', 'Conv03', 'Conv70', 'Conv71', 'Conv02', 'Conv80', 'Conv81', 'Conv01', 'Conv90', 'Conv91', 'Conv100']


23it [01:49,  4.74s/it]


w2tlsz06
['Conv10', 'Conv11', 'Conv20', 'Conv21', 'Conv30', 'Conv31', 'Conv40', 'Conv41', 'Conv50', 'Conv51', 'ConvRFF_04', 'Conv60', 'Conv61', 'ConvRFF_03', 'Conv70', 'Conv71', 'ConvRFF_02', 'Conv80', 'Conv81', 'ConvRFF_01', 'Conv90', 'Conv91', 'Conv100']


23it [01:45,  4.61s/it]


xqfrem5b
['Conv10', 'Conv11', 'Conv20', 'Conv21', 'Conv30', 'Conv31', 'Conv40', 'Conv41', 'Conv50', 'Conv51', 'ConvRFF_04', 'Conv60', 'Conv61', 'ConvRFF_03', 'Conv70', 'Conv71', 'ConvRFF_02', 'Conv80', 'Conv81', 'ConvRFF_01', 'Conv90', 'Conv91', 'Conv100']


...

19it [01:03,  3.34s/it]


qav9pmnj
['Conv10', 'Conv11', 'Conv20', 'Conv21', 'Conv30', 'Conv31', 'Conv40', 'Conv41', 'Conv50', 'Conv51', 'Conv60', 'Conv70', 'Pool30', 'Pool40']


14it [00:54,  3.86s/it]


04ilqh7l
['Res00_Conv00', 'Res00_Conv01', 'Res00_Conv02', 'Res01_Conv00', 'Res01_Conv01', 'Res01_Conv02', 'Res02_Conv00', 'Res02_Conv01', 'Res02_Conv02', 'Res03_Conv00', 'Res03_Conv01', 'Res03_Conv02', 'Res04_Conv00', 'Res04_Conv01', 'Res04_Conv02', 'Res05_Conv00', 'Res05_Conv01', 'Res05_Conv02', 'Res06_Conv00', 'Res06_Conv01', 'Res06_Conv02', 'Res07_Conv00', 'Res07_Conv01', 'Res07_Conv02', 'Res08_Conv00', 'Res08_Conv01', 'Res08_Conv02', 'Res09_Conv00', 'Res09_Conv01', 'Res09_Conv02', 'Res10_Conv00', 'Res10_Conv01', 'Res10_Conv02', 'Res11_Conv00', 'Res11_Conv01', 'Res11_Conv02', 'Res12_Conv00', 'Res12_Conv01', 'Res12_Conv02', 'Res13_Conv00', 'Res13_Conv01', 'Res13_Conv02', 'Res14_Conv00', 'Res14_Conv01', 'Res14_Conv02', 'Res15_Conv00', 'Res15_Conv01', 'Res15_Conv02', 'Res16_Conv00', 'Res16_Conv01', 'Res16_Conv02', 'Res17_Conv00', 'Res17_Conv01', 'Res17_Conv02', 'Res01_Act01', 'Res03_Act01', 'Res05_Act01', 'Res07_Act01']


53it [03:59,  5.22s/it]